##**Домашнее задание по теме "Улучшение качества модели"**

**Цель:**

Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

**Описание задания:**

В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

**Этапы работы:**

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)
2. Подготовьте датасет к обучению моделей:

- a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.
3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.
4. Обучите модель логистической регрессии с параметрами по умолчанию.
5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])
6. Оптимизируйте 3-4 параметра модели:
- a) Используйте GridSearchCV.
- b) Используйте RandomizedSearchCV.
- c) Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
- d) Повторите п. 5 после каждого итогового изменения параметров.
7. Сформулируйте выводы по проделанной работе:
- a) Сравните метрики построенных моделей.
- b) Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».


Текст оформляйте в отдельной ячейке Jupyter Notebook/Google Colab в формате markdown.
У графиков должен быть заголовок, подписи осей, легенда (опционально). Делайте графики бОльшего размера, чем стандартный вывод, чтобы увеличить читабельность.



**Решение:**

# 1. Получение данных и подготовка датасета

Для начала загрузим данные с сайта и посмотрим на первые строки датасета:

In [ ]:
import pandas as pd

# Загрузка CSV-файла
df = pd.read_csv('heart.csv')

In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


Пропусков в данных нет.

Датасет содержит атрибуты:

1. Возраст (Age) - возраст пациента в годах.
2. Пол (Sex) - пол пациента (M - мужской, F - женский).
3. Тип боли в груди (ChestPainType) - тип боли в груди (TA - типичная ангина, ATA - атипичная ангина, NAP - не-ангинальная боль, ASY - бессимптомный).
4. Кровяное давление в состоянии покоя (RestingBP) - кровяное давление в состоянии покоя в мм рт. ст.
5. Уровень холестерина (Cholesterol) - уровень холестерина в сыворотке крови в мм/дл.
6. Уровень сахара в крови натощак (FastingBS) - уровень сахара в крови натощак (1 - если уровень сахара в крови натощак > 120 мг/дл, 0 - в противном случае).
7. Результаты электрокардиограммы в состоянии покоя (RestingECG) - результаты электрокардиограммы в состоянии покоя (Normal - нормальный, ST - наличие ST-T волновой аномалии, LVH - указывающий на вероятную или определенную гипертрофию левого желудочка по критериям Estes).
8. Максимальная частота сердечных сокращений (MaxHR) - максимальная частота сердечных сокращений, достигнутая во время теста (числовое значение в диапазоне от 60 до 202).
9. Стенокардия при физической нагрузке (ExerciseAngina) - наличие или отсутствие стенокардии при физической нагрузке (Y - да, N - нет).
10. Значение старого пика (Oldpeak) - значение старого пика (ST) в делениях (числовое значение, измеренное в делениях).
11. Наклон пика ST при физической нагрузке (ST_Slope) - наклон пика ST при физической нагрузке (Up - восходящий, Flat - плоский, Down - нисходящий).

Целевая переменная в датасете - наличие или отсутствие сердечной болезни (HeartDisease) - принимает значения 1 (сердечная болезнь) или 0 (норма).

In [ ]:
data = df

Посмотрим соответствие данных указанным в атрибутах, и есть ли значения NaN.

In [ ]:
for column in data.select_dtypes(include=['object']).columns:
  nan_values = data[column].isnull().sum()  # Подсчет значений (NaN)
  unique_values = data[column].unique()     # Получение уникальных значений в столбце

# Ограничение вывода уникальных значений до первых 30
  unique_values = unique_values[:30]

  print(f"В столбце '{column}' есть {nan_values} пропущенных значений (NaN)")
  print(f"Уникальные значения в столбце '{column}': {unique_values}\n")

В столбце 'Sex' есть 0 пропущенных значений (NaN)
Уникальные значения в столбце 'Sex': ['M' 'F']

В столбце 'ChestPainType' есть 0 пропущенных значений (NaN)
Уникальные значения в столбце 'ChestPainType': ['ATA' 'NAP' 'ASY' 'TA']

В столбце 'RestingECG' есть 0 пропущенных значений (NaN)
Уникальные значения в столбце 'RestingECG': ['Normal' 'ST' 'LVH']

В столбце 'ExerciseAngina' есть 0 пропущенных значений (NaN)
Уникальные значения в столбце 'ExerciseAngina': ['N' 'Y']

В столбце 'ST_Slope' есть 0 пропущенных значений (NaN)
Уникальные значения в столбце 'ST_Slope': ['Up' 'Flat' 'Down']



Данные соответствуют описанию. Уникальные значения в каждом из столбцов соответсвуют описанию.

# 2. Подготовка данных к обучению моделей

## a) Для преобразования категориальных переменных в числовые значения воспользуемся методом pd.get_dummies:

In [ ]:
data = pd.get_dummies(data)

In [ ]:
data

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,False,True,False,...,False,False,False,True,False,True,False,False,False,True
1,49,160,180,0,156,1.0,1,True,False,False,...,True,False,False,True,False,True,False,False,True,False
2,37,130,283,0,98,0.0,0,False,True,False,...,False,False,False,False,True,True,False,False,False,True
3,48,138,214,0,108,1.5,1,True,False,True,...,False,False,False,True,False,False,True,False,True,False
4,54,150,195,0,122,0.0,0,False,True,False,...,True,False,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,False,True,False,...,False,True,False,True,False,True,False,False,True,False
914,68,144,193,1,141,3.4,1,False,True,True,...,False,False,False,True,False,True,False,False,True,False
915,57,130,131,0,115,1.2,1,False,True,True,...,False,False,False,True,False,False,True,False,True,False
916,57,130,236,0,174,0.0,1,True,False,False,...,False,False,True,False,False,True,False,False,True,False


# 3. Разделение выборки на обучающее и тестовое подмножество:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Обучение модели логистической регрессии с параметрами по умолчанию:

In [ ]:
from sklearn.linear_model import LogisticRegression

# Обучение модели логистической регрессии
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

# 5. Подсчет основных метрик модели:

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Оценка модели на тестовом множестве
y_pred_test = lr_model.predict(X_test)

accuracy_test = accuracy_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)

# Оценка модели на обучающем множестве
y_pred_train = lr_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)

print(f"Accuracy на обучающем множестве: {accuracy_train:.2f}")

print("Метрики на тестовом множестве:")
print(f"Accuracy: {accuracy_test:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy на обучающем множестве: 0.87
Метрики на тестовом множестве:
Accuracy: 0.85
Recall: 0.84
Precision: 0.90
F1-score: 0.87


Результаты метрик будем сохранять в сводную таблицу:

In [ ]:
# Добавляем результаты в таблицу
results = pd.DataFrame(columns=['Model', 'Train Accuracy', 'Test Accuracy', 'Recall', 'Precision', 'F1-score'])
results = pd.concat([results, pd.DataFrame({'Model': 'Logistic Regression',
                                           'Train Accuracy': accuracy_train,
                                           'Test Accuracy': accuracy_test,
                                           'Recall': recall,
                                           'Precision': precision,
                                           'F1-score': f1}, index=[0])], ignore_index=True)

results

<ipython-input-56-52fceb5ae377>:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({'Model': 'Logistic Regression',


,Model,Train Accuracy,Test Accuracy,Recall,Precision,F1-score
0,Logistic Regression,0.871935,0.853261,0.841121,0.9,0.869565


Модель логистической регрессии показала высокие значения метрик и хорошую способность к обобщению на тестовом наборе данных.

# 6. Оптимизация параметров модели:

## a) Используем GridSearchCV для поиска оптимальных параметров:

In [ ]:
from sklearn.model_selection import GridSearchCV

# Обновление параметров модели логистической регрессии с увеличенным числом итераций
lr_model = LogisticRegression(max_iter=2000, C=1, solver='lbfgs')

# Обучение модели
lr_model.fit(X_train, y_train)

# Создание объекта GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}

grid_search = GridSearchCV(estimator=lr_model, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

best_lr_model = grid_search.best_estimator_

# Оценка модели на тестовом множестве
y_pred_test = best_lr_model.predict(X_test)

accuracy_test = accuracy_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)

# Оценка модели на тренировочном множестве
y_pred_train = best_lr_model.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)

print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Accuracy на обучающем множестве у лучшей модели: {accuracy_train:.2f}")

print("Метрики качества у лучшей модели:")
print(f"Accuracy: {accuracy_test:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1-score: {f1:.2f}")

# Добавляем результаты в таблицу
results = pd.concat([results, pd.DataFrame({'Model': 'Logistic Regression (Optimized)',
                                           'Train Accuracy': accuracy_train,
                                           'Test Accuracy': accuracy_test,
                                           'Recall': recall,
                                           'Precision': precision,
                                           'F1-score': f1}, index=[0])], ignore_index=True)

results

Лучшие параметры: {'C': 0.01, 'solver': 'lbfgs'}
Accuracy на обучающем множестве у лучшей модели: 0.87
Метрики качества у лучшей модели:
Accuracy: 0.86
Recall: 0.85
Precision: 0.91
F1-score: 0.88


,Model,Train Accuracy,Test Accuracy,Recall,Precision,F1-score
0,Logistic Regression,0.871935,0.853261,0.841121,0.90,0.869565
1,Logistic Regression (Optimized),0.870572,0.864130,0.850467,0.91,0.879227


Оптимизированная модель логистической регрессии показала улучшение производительности по сравнению с исходной моделью, в частности, более высокую точность, полноту, precision и F1-score на тестовом наборе данных. Это указывает на улучшенную способность модели обобщать и предсказывать результаты на новых данных.

Эти улучшения могли быть достигнуты за счет оптимизации параметров модели через GridSearchCV (изменение параметров C и solver), что позволило найти наилучшую модель с учетом заданных критериев.

## b) Используем RandomizedSearchCV и добавление нескольких моделей классификации с вариациями параметров

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Модели классификации
models = {
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

# Параметры для RandomizedSearchCV
params = {
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10, 20], 'min_samples_split': [2, 5, 10]},
    'Support Vector Machine': {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale'], 'kernel': ['linear', 'rbf']},
    'Decision Tree': {'max_depth': [None, 5, 10, 20], 'min_samples_split': [2, 5, 10]},
    'K-Nearest Neighbors': {'n_neighbors': [3, 5, 10], 'leaf_size': [30, 50, 100]}
}

# Обработка каждой модели
for model_name, model in models.items():
    rscv = RandomizedSearchCV(estimator=model, param_distributions=params[model_name],
                              n_iter=9, scoring='accuracy', cv=5)
    rscv.fit(X_train, y_train)

    best_model = rscv.best_estimator_
    y_pred_test = best_model.predict(X_test)

    accuracy_test = accuracy_score(y_test, y_pred_test)
    recall = recall_score(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test)
    f1 = f1_score(y_test, y_pred_test)

    y_pred_train = best_model.predict(X_train)
    accuracy_train = accuracy_score(y_train, y_pred_train)

    print(f"Лучшие параметры для {model_name}: {rscv.best_params_}")

    # Добавление результатов в таблицу results для каждой модели
    results = pd.concat([results, pd.DataFrame({'Model': model_name + ' (Optimized)',
                                                'Train Accuracy': accuracy_train,
                                                'Test Accuracy': accuracy_test,
                                                'Recall': recall,
                                                'Precision': precision,
                                                'F1-score': f1}, index=[0])], ignore_index=True)
results

Лучшие параметры для Random Forest: {'n_estimators': 200, 'min_samples_split': 5, 'max_depth': None}
Лучшие параметры для Support Vector Machine: {'kernel': 'linear', 'gamma': 'scale', 'C': 0.1}
Лучшие параметры для Decision Tree: {'min_samples_split': 2, 'max_depth': 5}
Лучшие параметры для K-Nearest Neighbors: {'n_neighbors': 5, 'leaf_size': 30}


,Model,Train Accuracy,Test Accuracy,Recall,Precision,F1-score
0,Logistic Regression,0.871935,0.853261,0.841121,0.900000,0.869565
1,Logistic Regression (Optimized),0.870572,0.864130,0.850467,0.910000,0.879227
2,Random Forest (Optimized),0.967302,0.875000,0.887850,0.896226,0.892019
3,Support Vector Machine (Optimized),0.877384,0.858696,0.850467,0.900990,0.875000
4,Decision Tree (Optimized),0.897820,0.864130,0.859813,0.901961,0.880383
5,K-Nearest Neighbors (Optimized),0.885559,0.858696,0.841121,0.909091,0.873786


Можно сделать следующие выводы:
- Наилучшие метрики точности показала модель Модель Random Forest после оптимизации параметров.
- Остальные модели также улучшили свою производительность после оптимизации.

## Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

Stacking Classifier в ДЗ "Ансамблирование" показывал следующие результаты:
   - Train Set:
     - Accuracy: 0.90
     - Precision: 0.91 (класс 0), 0.90 (класс 1)
     - Recall: 0.87 (класс 0), 0.93 (класс 1)
     - F1-score: 0.89 (класс 0), 0.91 (класс 1)

   - Test Set:
     - Accuracy: 0.87
     - Precision: 0.83 (класс 0), 0.90 (класс 1)
     - Recall: 0.87 (класс 0), 0.87 (класс 1)
     - F1-score: 0.85 (класс 0), 0.89 (класс 1)

Выводы:

1. Точность (accuracy) модели Stacking Classifier на тренировочных данных составляет 0.90, в то время как у лучшей модели Random Forest из текущего задания она была равна 0.97. Это указывает на то, что Random Forest показал более высокую точность на тренировочных данных.

2. Точность (accuracy) модели Stacking Classifier на тестовых данных составляет 0.87, в то время как у лучшей модели Random Forest из текущего задания она была равна 0.88. Обе модели показали примерно одинаковую точность на тестовых данных.

3. Метрики recall, precision и F1-score у Stacking Classifier также близки к лучшим результатам, полученным у других моделей из текущего задания.